# Lab Parsing Data
worked with Haimant

iNaturalist API

Select Species: Green Anole
taxon_id=36514

Select Geographic Area: Miami, FL
place_id=2345

Create Dataframe with JSON results

In [50]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

url = 'https://api.inaturalist.org/v1/observations/?taxon_id=36514&place_id=2345'
# url = 'https://api.inaturalist.org/v1/observations?verifiable=true&order_by=observations.id&order=desc&page=1&spam=false&place_id=2345&taxon_id=36514&locale=en-US&per_page=24&return_bounds=true'

response = requests.get(url)
result = response.json()
print(result)

{'total_results': 1061, 'page': 1, 'per_page': 30, 'results': [{'out_of_range': False, 'quality_grade': 'research', 'time_observed_at': '2019-09-27T11:41:57-04:00', 'taxon_geoprivacy': 'open', 'annotations': [], 'context_user_geoprivacy': None, 'uuid': '3fc1ce8c-2213-4d27-8d1b-dca6f01be20e', 'observed_on_details': {'date': '2019-09-27', 'week': 39, 'month': 9, 'hour': 11, 'year': 2019, 'day': 27}, 'id': 33483335, 'cached_votes_total': 0, 'identifications_most_agree': True, 'created_at_details': {'date': '2019-09-27', 'week': 39, 'month': 9, 'hour': 11, 'year': 2019, 'day': 27}, 'species_guess': 'Green Anole', 'identifications_most_disagree': False, 'tags': [], 'positional_accuracy': 5, 'comments_count': 0, 'site_id': 1, 'created_time_zone': 'America/New_York', 'id_please': False, 'license_code': 'cc-by-nc', 'observed_time_zone': 'America/New_York', 'quality_metrics': [], 'public_positional_accuracy': 5, 'reviewed_by': [85164, 268311, 971085, 2156541], 'context_geoprivacy': None, 'oauth

In [53]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from math import ceil

url = f'https://api.inaturalist.org/v1/observations/?taxon_id=36514&place_id=2345&per_page=200&page={i}'

response = requests.get(url)
result = response.json()

totalr = result['total_results']

perp = result['per_page']

pages = ceil(totalr / perp)
print(pages)

6


In [70]:
page_data = []
for i in range(1, pages + 1):
    url = f'https://api.inaturalist.org/v1/observations/?taxon_id=36514&place_id=2345&per_page=200&page={i}'
    response = requests.get(url)
    results = response.json()['results']
    flat_results = json_normalize(results)
#     results = json_normalize(response.json()['results']) # DJK combined my two lines
    page_data.append(pd.DataFrame(flat_results))

ga_df = pd.concat(page_data) # combine all the dataframes that are in this list of df's i'm passing in
print(ga_df.head())


                                         annotations  cached_votes_total  \
0                                                 []                   0   
1                                                 []                   0   
2                                                 []                   0   
3  [{'user_id': 268311, 'concatenated_attr_val': ...                   0   
4                                                 []                   0   

   captive comments  comments_count  community_taxon_id context_geoprivacy  \
0    False       []               0             36514.0               None   
1    False       []               0             36514.0               None   
2    False       []               0             36514.0               None   
3    False       []               0             36514.0               None   
4    False       []               0             36514.0               None   

  context_taxon_geoprivacy context_user_geoprivacy                 created

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [71]:
print(ga_df.dtypes)

annotations                     object
cached_votes_total               int64
captive                           bool
comments                        object
comments_count                   int64
community_taxon_id             float64
context_geoprivacy              object
context_taxon_geoprivacy        object
context_user_geoprivacy         object
created_at                      object
created_at_details.date         object
created_at_details.day           int64
created_at_details.hour          int64
created_at_details.month         int64
created_at_details.week          int64
created_at_details.year          int64
created_time_zone               object
description                     object
faves                           object
faves_count                      int64
flags                           object
geojson.coordinates             object
geojson.type                    object
geoprivacy                      object
id                               int64
id_please                

Analysis:
Total number of sightings

In [74]:
print("The total number of sightings is: " + str(len(ga_df['user.id'])))

The total number of sightings is: 1061


Which user has reported the most sightings?

In [118]:
# print("The user with the most sightings is: " + str(ga_df['user.login'].max())) # returns an incorrect user
# print(ga_df['user.login'].value_counts())
most_sightings = ga_df['user.login'].value_counts()
# print("The user with the most sightings is: " + str(ga_df['user.login'].value_counts().max()) + " with this many sightings: " + str(ga_df['user.login'].value_counts().max()))
print(most_sightings.head())

print("The user with the most sightings is: " + str(most_sightings[0]))

#DJK
# df['user.login'].value)counts()



josemartimastms         97
palmettomiddleschool    65
bennypoo                36
jfk727                  31
ochoaswatchers          30
Name: user.login, dtype: int64
The user with the most sightings is: 97


Which month has the most sightings?
make a new column, putting in the month, and then the month with the largest count.

In [94]:
import calendar
ga_df['MonthName'] = [calendar.month_name[int(x)] for x in ga_df['observed_on_details.month'].notnull()]
# print(ga_df['observed_on_details.month'].head())
# print(calendar.month_name[9])

# for x in ga_df['observed_on_details.month'].head():
#     print(calendar.month_name[int(x)])


# print(ga_df['MonthName'].head())
# print(ga_df['MonthName'].describe())

print("The month with the most sightings is: " + str(ga_df['MonthName'].describe().top))

# df['observedondetails.month'].apply(lambda x: month_name[x]) # from DJK, a lambda instead of my for-loop

The month with the most sightings is: January


Bonus: Search within a boundary box